In [ ]:
!export CUDA_VISIBLE_DEVICES=2 ## 2번 GPU 사용 (모델이 깊어지면 지정하여 전체 사용GPU 사용분산)

In [ ]:
## CNN 모델사용 

In [10]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt


In [11]:
tf.reset_default_graph()

def parser(serialized_example):
    feature = {
        'age': tf.FixedLenFeature([1], tf.int64),#Configuration for parsing a fixed-length input feature.
        'img': tf.FixedLenFeature([61*49], tf.int64) ## 61*49 픽셀
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, feature)
    age = tf.cast(parsed_feature['age'], tf.int32)## one-hot 으로 바꿔야 하므로 intiger
    img = tf.cast(parsed_feature['img'], tf.float32)## imgage는 실수형
    
    return age, img

In [12]:
face_train = './cnn_dataset/face_train.tfrecord'
face_test = './cnn_dataset/face_test.tfrecord'
train_dataset = tf.contrib.data.TFRecordDataset(face_train).map(parser)## parcer fuction mapping 
train_dataset = train_dataset.batch(32).shuffle(7777) 

In [13]:
test_dataset = tf.contrib.data.TFRecordDataset(face_test).map(parser)
test_dataset = test_dataset.batch(32).shuffle(7777)


In [ ]:
#itr = train_dataset.make_one_shot_iterator() ## iterator 생성
itr = tf.contrib.data.Iterator.from_structure(
    train_dataset.output_types, train_dataset.output_shapes) ## type, shape만 정하면 됨

age, img = itr.get_next()## 하나끝나면 그다음꺼 하나씩 빼옴 

In [ ]:
## one_shot_iterator 문제점: trainset, testset의 크기가 다른경우 
## out of range 에러가 생기는 경우 - 해결: initializing 

img = tf.reshape(img,[-1,61,49,1]) ## 자동셋팅, 행61 * 열49 pixel, 1color(흑백) 
img = tf.cast(img, tf.float32) ## 실수형 32 변경 

In [ ]:
age = tf.reshape(age,[-1]) ## 자동셋팅 일자로 쫙펴줌 ## reduce_mean 써도됨
age_onehot = tf.one_hot(age, depth = 3, axis = -1, dtype = tf.float32)

## trainset, testset 하나의 itr로 연동 
training_init_op = itr.make_initializer(train_dataset)
test_init_op = itr.make_initializer(test_dataset)

In [ ]:
## convolution 시작 

In [ ]:
def model(x, activation, dropout_prob, reuse = False):
    
    conv1 = tf.layers.conv2d(img,filters=16,
                             kernel_size = 4, padding='same', 
                             activation = activation, reuse =reuse, 
                             name = 'conv1')
    pool1 = tf.layers.max_pooling2d(conv1, pool_size = 2, strides =2) ## 사이즈2, 보폭2 인 polling

    conv2= tf.layers.conv2d(conv1, filters=32,
                           kernel_size = 3, padding='same', 
                            activation = activation, reuse =reuse, 
                             name = 'conv2')
    pool2 = tf.layers.max_pooling2d(conv2,pool_size = 2, strides =2) ## shape 가 홀수인경우 pooling 하면 나머지줄은 버림 
    
    conv3= tf.layers.conv2d(conv2, filters=32,
                           kernel_size = 4, padding='same', 
                            activation = activation, reuse =reuse, 
                             name = 'conv3')

    pool3 = tf.layers.max_pooling2d(conv3,pool_size = 2, strides =2)


    ## layer 시작 세팅 
    flat = tf.layers.flatten(pool3)
    ## flat_size = int(pool3.shape[1]) * int(pool3.shape[2]) * int(pool3.shape[3])
    ## flat = tf.reshape(pool3, [-1, flat_size]) 으로 대체가능 

    dropout1 = tf.layers.dropout(flat, dropout_prob)
    fc1 = tf.layers.dense(flat, units = 5000, 
                          activation = tf.nn.relu, 
                          reuse =reuse, name = 'fc1')

    dropout2 = tf.layers.dropout(fc1, dropout_prob)
    fc2 = tf.layers.dense(fc1, units = 1000, 
                          activation = tf.nn.relu, 
                          reuse =reuse, name = 'fc2')

    out = tf.layers.dense(fc2, units = 3, 
                          activation = tf.nn.relu, 
                          reuse =reuse, name = 'out')

    return out 

In [ ]:
train_out = model(img, tf.nn.relu, 0.7) ## dropout은 70% 
test_out = model(img, tf.nn.relu, 0, True) ## drop out은 0% 

In [ ]:
loss = tf.losses.softmax_cross_entropy(age_onehot, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-6).minimize(loss)

In [ ]:
tf.summary.scalar ('loss', loss) ## loss 에 대해서 summary collecting
merged = tf.summary.merge_all()

In [ ]:
pred = tf.argmax(tf.nn.softmax(test_out), axis=1)
accuracy = tf.metrics.accuracy(age, pred)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    #writer = tf.summary.FileWriter('cnn_dataset/'.sess.graph)
    for epoch in range(10):
        sess.run(training_init_op)
        for train_step in range(999999):
            try:
                _, _loss = sess.run([train_out, loss])
                _summ = sess.run(merged)
            except tf.errors.OutOfRangeError:
                break
                
        sess.run(test_init_op)
        for test_step in range(9999999):
            try:
                _acc = sess.run(accuracy)
                    
            except tf.errors.OutOfRangeError:
                break
                
        print('epochs: {}, loss: {}, acc: {}'.format(epoch, _loss, _acc[0]))

    # saver.save(sess,'cnn_dataset parameters')